In [215]:
from src.data_processing import data_processing, data_dropping
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, make_scorer, accuracy_score
from nltk.stem import WordNetLemmatizer as Lemmatizer

In [5]:
with open('data/predictor.p', 'rb') as file:
    predictor = pickle.load(file)
with open('data/target.p', 'rb') as file:
    target = pickle.load(file)

In [8]:
pred = data_processing(data_dropping(predictor))

In [13]:
pred = pred[['existing_stories', 'existing_use', 'proposed_use', 'description']]

In [14]:
pred['has_existing_stories'] = (pred['existing_stories'].fillna(0) > 0).astype(int)

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
pred[['existing_use', 'proposed_use', 'description']] = pred[['existing_use', 'proposed_use', 'description']].fillna('').astype(str)

In [28]:
pred['text'] = pred[['existing_use', 'proposed_use', 'description']].apply(lambda x: ' '.join(x), axis = 1)

C:\Users\pengj\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
target.columns

Index(['fires_before', 'fires_after', 'date', 'fire_rate_after',
       'fire_rate_before'],
      dtype='object')

In [30]:
with open('data/pred_trimmed.p', 'wb') as file:
    pickle.dump(pred, file)

In [221]:
text = pred['text'].values

In [192]:
y_full = (target['fire_rate_after'] > target['fire_rate_before']).astype(int).values

In [229]:
text = np.array([' '.join([Lemmatizer.lemmatize('n', word.lower()) for word in doc.split()]) for doc in text])

In [231]:
text

array(['2 family dwelling 2 family dwelling renew for 200112175408 & 200105290149 for final inspection nan',
       'apartment apartment correct violation in report of physical inspection application no 3r-3956 dec 18,2003 nan',
       'apartment apartment correct violation in report of physical inspection application no 3r-3956 dec 18,2003 nan',
       ...,
       '2 family dwelling 2 family dwelling to obtain a final inspection for work approved under previus permit 201402289608 & 201704123763. all work is complete.',
       '2 family dwelling 2 family dwelling to obtain a final inspection for work approved under previus permit 201402289608 & 201704123763. all work is complete.',
       '1 family dwelling 1 family dwelling drywall replacement in 3 bedroom and 1 hallway. also add insulation a needed'],
      dtype='<U347')

In [182]:
?

In [233]:
X = text[sample_idx]
y = y_full[sample_idx]

In [234]:
# train = np.full(n, False)
# train_idx = np.random.choice(n, int(.75*n), replace = False)
# for idx in train_idx:
#     train[idx] = True
# test = ~train

In [235]:
vec = TfidfVectorizer(stop_words = 'english')

In [236]:
X = vec.fit_transform(X)

In [197]:
model = LogisticRegression(penalty = 'l1')

In [198]:
model.fit(X[train],y[train])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [199]:
model.score(X[test], y[test])

0.866

In [200]:
roc_auc_score(y[test], model.predict_proba(X[test])[:,1])

0.7829562411776567

In [201]:
def f1(y_test, y_pred_prob, threshold):
    y_pred = (y_pred_prob > threshold).astype(int)
    return f1_score(y_test, y_pred)

In [202]:
current_best = 0
best_i = 0
for i in range(1000):
    f = f1(y[test], model.predict_proba(X[test])[:,1], i/1000)
    if f > current_best:
        current_best = f
        best_i = i
    

C:\Users\pengj\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [203]:
model.coef_

array([[0., 0., 0., ..., 0., 0., 0.]])

In [204]:
f1(y[test], model.predict_proba(X[test])[:,1], best_i/1000)

0.46418338108882523

In [205]:
vec.vocabulary_;

In [206]:
vocab_idx = {v: k for k, v in vec.vocabulary_.items()}

In [207]:
important_words = {}

In [208]:
for i, coef in enumerate(model.coef_[0]):
    if coef > 0:
        important_words[vocab_idx[i]] = coef

In [211]:
important_words

{'155': 3.1035912135296058,
 '8th': 0.5653977168600285,
 'add': 1.3189284794110383,
 'apt': 4.463987721335684,
 'bathroom': 0.777527495715587,
 'ceiling': 2.4061458459505816,
 'closet': 0.20693889987998118,
 'common': 0.2572805212635152,
 'drawings': 1.3067239246464866,
 'emergency': 0.33072454563355463,
 'faced': 1.1433798692146413,
 'finishes': 3.6574502626583008,
 'fixtures': 0.5123874065592303,
 'flr': 0.1328012300893051,
 'include': 1.1432906942533385,
 'interior': 1.2391021529517692,
 'kitchen': 0.8159770238906059,
 'like': 1.3433312672515416,
 'lobby': 2.0466061790820875,
 'maher': 3.945060653556438,
 'mep': 2.2384294066448924,
 'motel': 0.29619647229672375,
 'non': 0.19537872737189085,
 'obtain': 1.7005488112504799,
 'office': 0.5801552681465681,
 'ordinance': 0.20006497201715248,
 'pa': 0.7260685567025665,
 'pendents': 1.0717323310678821,
 'permit': 0.28628134263447824,
 'recreation': 1.8134444342215625,
 'ref': 2.7855127302584353,
 'remodeling': 1.9617031284357653,
 'req': 0.